In [1]:
import numpy as np
import pandas as pd
import models as mdl
import utility as util
import constraint as cnst
import L0 as lreg
import torch
from torch import optim
from torch import nn
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
torch.manual_seed(0)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs=8000 #5500
batch_size=500
hid_y=100
beta=1#0.03,0.5
BCE=nn.BCELoss()
MSE=nn.MSELoss(reduction='mean')
MSE_2=nn.MSELoss(reduction='sum')

In [3]:
total_loss=[]
tr_pehe=[]
val_pehe=[]
reg_loss_tr=[]
classT_loss_tr=[]
classW_loss_tr=[]
wasser_loss_tr=[]
rc_loss=[]
OR_reg_li=[]

val_reg_li=[]
val_tloss_li=[]
val_wasser_li=[]
val_rec_li=[]
kl_l=[]
snr_gamma=[]
snr_delta=[]
snr_upsilon=[]
snr_omega=[]

gamma_mean=[]
gamma_var=[]
delta_mean=[]
delta_var=[]
upsilon_mean=[]
upsilon_var=[]
omega_mean=[]
omega_var=[]
total_mean=[]
total_var=[]
lambd_li=[]
kl_total=[]
pen_li=[]
gamma_count=[]
delta_count=[]
upsilon_count=[]
omega_count=[]
mse_li=[]
l0_li=[]
lambd2_li=[]
def train_CFR(x_data,y_data,t):
   
    torch.manual_seed(0)
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    patience = 500 # Number of epochs without improvement before stopping
    max_saved_models = 7  # Store the best 10 models
    
    # Tracking best models (min-heap)
    best_models = []
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    saved_model_count=0
    tol=t
    
    
    iter_num = 0
    alpha=0.99
    pretrain=1
    lbd_step=30
    kl_flag=True

    #x_data,y_data=get_data('train',1)
    X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,
                                       random_state=42, 
                                       test_size=0.2, shuffle=True, stratify=x_data.iloc[:,0].values)
    # write input_feature instead of 25
    net=TarNet(input_features,hid_enc,lat_dim,.1)
    #wnet=Wclassifier(lat_dim,.1)
    tnet=Tclassifier(lat_dim*2,.1)
    rnet=Regressors(lat_dim*2,hid_y,.1)
    mse_constraint=Constraint(tol, device, lambda_min=0.0, lambda_max=200.0, lambda_init=1.0, alpha=0.99)
    #dnet=Decoder_VAE(lat_dim*4,input_features,.1)
    dnet=Decoder(lat_dim*4,input_features,.1)
    sparse_model1 = SparseNet(lat_dim,1)
    sparse_model2 = SparseNet(lat_dim,1)
    sparse_model3 = SparseNet(lat_dim,1)
    sparse_model4 = SparseNet(lat_dim,1)
    net.to(device)
    tnet.to(device)
    rnet.to(device)
    dnet.to(device)
    sparse_model1.to(device)
    sparse_model2.to(device)
    sparse_model3.to(device)
    sparse_model4.to(device)
    #dnet=Decoder(700,input_features,.1)
    #print(input_features)
    #rnet=Regressors_2(lat_dim*2,.1)
    opt_net = torch.optim.Adam(net.parameters(), lr=0.00005,weight_decay=1e-3)#1e-4
    opt_nett = torch.optim.Adam(tnet.parameters(), lr=0.00008, weight_decay=1e-3) #00008
    opt_netr = torch.optim.Adam(rnet.parameters(), lr=0.00008,weight_decay=1e-3) #0.0008
    opt_netd = torch.optim.Adam(dnet.parameters(), lr=0.00002,weight_decay=1e-3) #,weight_decay=1e-4
    opt_lmd = torch.optim.Adam(mse_constraint.parameters(), lr=0.004,weight_decay=1e-3)
    sparse_optimizer1 = optim.Adam(sparse_model1.parameters(), lr=0.003) #0.01
    sparse_optimizer2 = optim.Adam(sparse_model2.parameters(), lr=0.003) #0.01
    sparse_optimizer3 = optim.Adam(sparse_model3.parameters(), lr=0.001) #0.01
    sparse_optimizer4 = optim.Adam(sparse_model4.parameters(), lr=0.003) #0.01
    
   
    
    
    mask_gamma = torch.ones(dim_range*4).to(device)
    mask_delta = torch.ones(dim_range*4).to(device)
    mask_upsilon = torch.ones(dim_range*4).to(device)
    mask_omega = torch.ones(dim_range*4).to(device)
    milestones = {int(epochs * ratio): False for ratio in [0.99]} #0.4, 0.6, 0.99

    for ep in tqdm(range(1,epochs+1 )):
 
       
        train_dataloader_sr, train_dataloader_tr=get_dataloader(X_train,y_train,batch_size)
        tot_los=0
        reg_los=0
        t_los=0
        w_los=0
        wa_los=0
        rc_los=0
        rereg_los=0
        cnt=0
        kl=0
        kl_t=0
        mse_er=0
        pen=0
        gc=0
        dc=0
        uc=0
        oc=0
        l0=0
 
        for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):
            
            xs,ys=train_source_data
            xt,yt=train_target_data
            # Replace 30 with :
            xs_train=xs[:,5:].to(device)
            xt_train=xt[:,5:].to(device)
            ys.to(device)
            yt.to(device)
            train_x=torch.cat((xs_train,xt_train),0)
            train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1).to(device)
            true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1).to(device)
            concat_true=torch.cat((train_y,true_t),1)
            prop_t1=(xt_train.shape[0]/train_x.shape[0])
            phi, phi_mean, phi_var=net(train_x)

            
                


            
            phi_gamma=phi[:,fstart:fend]
            phi_delta=phi[:,sstart:send]
            phi_upsilon=phi[:,tstart:tend]
            phi_irr=phi[:,frstart:frend]
            
      
            masked_phi_gamma, penalty_gamma,mask_gamma_d = sparse_model1(phi)
            masked_phi_delta, penalty_delta,mask_delta_d = sparse_model2(phi)
            masked_phi_upsilon, penalty_upsilon,mask_upsilon_d = sparse_model3(phi)
            masked_phi_omega, penalty_omega,mask_omega_d = sparse_model4(phi)
           
            mask_gamma=mask_gamma_d.clone()
            mask_delta=mask_delta_d.clone()
            mask_upsilon=mask_upsilon_d.clone()
            mask_omega=mask_omega_d.clone()
            
            gamma_c=get_dim_count(mask_gamma,0.5)
            delta_c=get_dim_count(mask_delta,0.5)
            upsilon_c=get_dim_count(mask_upsilon,0.5)
            omega_c=get_dim_count(mask_omega,0.5)
            
            penalty=penalty_delta+penalty_upsilon+penalty_gamma#+penalty_omega++
            
            

            phi_all=torch.cat((masked_phi_gamma,masked_phi_delta,masked_phi_upsilon,masked_phi_omega), 1)
            del_ups=torch.cat((masked_phi_delta, masked_phi_upsilon), 1)
            gam_del=torch.cat((masked_phi_gamma,masked_phi_delta), 1)
            
            
            concat_pred=rnet(del_ups)
            #w_f=wnet(phi_delta)
            w_t=tnet(gam_del)
            decoded_space=dnet(phi_all)
            
            predicted_y=torch.unsqueeze(torch.where(true_t.squeeze() == 0, concat_pred[:,0], concat_pred[:,1]),dim=1)
           
            opt_net.zero_grad()
           
            opt_nett.zero_grad()
            opt_netr.zero_grad()
            opt_netd.zero_grad()
            sparse_optimizer1.zero_grad()
            sparse_optimizer2.zero_grad()
            sparse_optimizer3.zero_grad()
            sparse_optimizer4.zero_grad()
            opt_lmd.zero_grad()
            
           
            Reconstruction_loss=(MSE(decoded_space[:,0:6],train_x[:,0:6])+BCE(decoded_space[:,6:25],train_x[:,6:25])+MSE(decoded_space[:,25:],train_x[:,25:]))
            #Reconstruction_loss=MSE(decoded_space,train_x)
            y_LOSS,mse_cons=Y_GECO(predicted_y,train_y, tol)
            #Reconstruction_loss=torch.mean(lambd*rc_LOSS)
            mask=torch.cat((mask_gamma,mask_delta,mask_upsilon),1)
           
            #kl_div=kl_loss_2(phi_mean*mask, phi_var*mask,beta,kl_flag)
            kl_div_g=kl_loss_2(phi_mean*mask_gamma, phi_var*mask_gamma,beta,kl_flag)
            kl_div_d=kl_loss_2(phi_mean*mask_delta, phi_var*mask_delta,beta,kl_flag)
            kl_div_u=kl_loss_2(phi_mean*mask_upsilon, phi_var*mask_upsilon,beta,kl_flag)
            kl_div_o=kl_loss_2(phi_mean*mask_omega, phi_var*mask_omega,beta,kl_flag)
            kl_div=((kl_div_g+kl_div_d+kl_div_u+kl_div_o)/4)
            
            Wassloss,dist=wasserstein(masked_phi_upsilon,true_t)
           
            Tloss=BCE(w_t, true_t)
            cnt=cnt+1
            
            
            
           
            Rloss,lambd=mse_constraint(y_LOSS)

            combined_loss=(Rloss)+(Wassloss)+(Tloss)+(Reconstruction_loss)+(2*kl_div)\
            +(0.04*penalty_delta)+(0.00001*penalty_upsilon)+(0.0002*penalty_gamma)+(0.0001*penalty_omega)
            
            
            
            
            combined_loss.backward()
            tot_los=tot_los+combined_loss.item()
            reg_los=reg_los+Rloss.item()
            t_los=t_los+Tloss.item()
            #w_los=w_los+OR.item()
            wa_los=wa_los+Wassloss.item()
            rc_los=rc_los+Reconstruction_loss.item()
            #rereg_los=rereg_los+OR_re.item()
            kl=kl+kl_div.item()
            kl_t=kl_t+torch.mean(kl_loss_3(phi_mean, phi_var,1)).item()
            mse_er=mse_er+mse_cons
            pen=pen+penalty.item()
            gc=gc+gamma_c
            dc=dc+delta_c
            uc=uc+upsilon_c
            oc=oc+omega_c
            l0=l0+(penalty_delta.detach().item()+penalty_upsilon.detach().item()+penalty_gamma.detach().item())
            
            opt_net.step()
           
            opt_nett.step()
            opt_netr.step()
            opt_netd.step()
            sparse_optimizer1.step()
            sparse_optimizer2.step()
            sparse_optimizer3.step()
            opt_lmd.step()
            sparse_optimizer4.step()

           

       
          
        
        val_PEHE_nn=cal_pehe_nn(X_val, y_val,rnet,net,mask_gamma,mask_delta,mask_upsilon)
        tr_PEHE=cal_pehe(X_train,y_train,rnet,net,mask_gamma,mask_delta,mask_upsilon)


         
        # Evaluation of validation
        X_train_t= X_train.to_numpy()
        X_train_t=torch.from_numpy(X_train_t.astype(np.float32)).to(device)
        
        X_val_t=X_val.to_numpy()
        X_val_t=torch.from_numpy(X_val_t.astype(np.float32)).to(device)
        
        y_val_t=y_val.to_numpy()
        y_val_t=torch.from_numpy(y_val_t.astype(np.float32)).to(device)

       

    

        if ep > 2000:
                if val_PEHE_nn < best_val_loss:
                    best_val_loss = val_PEHE_nn
                    epochs_without_improvement = 0  # Reset patience counter
                else:
                    epochs_without_improvement += 1
                
                # Save the model if it's among the top 10 and hasn't improved for 50 epochs
                if epochs_without_improvement >= patience:
                    epochs_without_improvement = 0 
                    if len(best_models) < max_saved_models or val_PEHE_nn < best_models[-1][0]:
                        # Save the model
                        model_state = {
                            'encoder': net.state_dict(),
                            'regressor': rnet.state_dict(),
                            'classifier': tnet.state_dict(),
                            'val_loss': val_PEHE_nn,
                            'epoch': ep
                        }
                        model_filename = f'Geco_1/model_{ep}_val{val_PEHE_nn:.4f}.pth'
                        torch.save(model_state, model_filename)
        
                        # Add to best models list
                        heapq.heappush(best_models, (val_PEHE_nn, model_filename))
                        saved_model_count += 1
        
                        # Keep only the best 10 models
                        if len(best_models) > max_saved_models:
                            worst_model = heapq.heappop(best_models)  # Remove the worst model
                            torch.cuda.empty_cache()  # Free memory if needed
        
                # Stop training after saving 10 models
        if saved_model_count >= max_saved_models:
                print(f"Early stopping at epoch {ep}: 10 best models saved.")
                break
       
           
        
        reg_loss_tr.append(reg_los/cnt)
        classT_loss_tr.append(t_los/cnt)
        classW_loss_tr.append(w_los/cnt)
        wasser_loss_tr.append(wa_los/cnt)
        total_loss.append(tot_los/cnt)
        tr_pehe.append(tr_PEHE)
        val_pehe.append(val_PEHE_nn)
        #print(rc_los)
        rc_loss.append(rc_los/cnt)
        OR_reg_li.append(rereg_los)
        kl_l.append(kl)
        lambd_li.append(lambd)
        kl_total.append(kl_t)
        pen_li.append(pen/cnt)
        gamma_count.append(gc/cnt)
        delta_count.append(dc/cnt)
        upsilon_count.append(uc/cnt)
        omega_count.append(oc/cnt)
        mse_li.append(mse_er/cnt)
        l0_li.append(l0/cnt)
        

    return rnet,net,X_train_t,mask_gamma,mask_delta,mask_upsilon,mask_omega,best_models



In [4]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

lat_dim_li=[10*4] #
dummy_list=[0,5,10,15]# 
tol_li=[0.4] 
gamma_dim_count=[]
delta_dim_count=[]
upsilon_dim_count=[]
omega_dim_count=[]
column_name = ['0', '5', '10', '15','20']
for ldi,ld in enumerate(lat_dim_li):
    dim_range=ld//4 #5,10,15
    lat_dim=dim_range*4
    fstart=0
    fend=fstart+dim_range
    sstart=fend
    send=sstart+dim_range
    tstart=send
    tend=tstart+dim_range
    frstart=tend
    frend=frstart+dim_range
    lat_dim=ld
    hid_enc=ld
    print('latent dimensions', lat_dim)
    df = pd.DataFrame(list(),columns=column_name)
        # writing CSV
    df.to_csv(f"IHDP_Results/OURS_PEHE_{ld}.csv", index=False)
    #for di,d in enumerate(dummy_list):
    for di,d in enumerate(dummy_list):
    
        
        pehe_l=[]
        dummies=d
        pehe_final=[]
        input_features=25+dummies
        kl_avg=[]
        reg_loss_li=[]
        tol=0.4
        
        for i in range(1,31):
            #clean()
            x_data_tr,y_data_tr=get_data('train',i)
       
            x_data_n=add_dummy_features_shuffle(x_data_tr,dummies)
           
            x_data_n_nor= pd.DataFrame(scaler_x.fit_transform(x_data_n.iloc[:,5:]))
            data_train_tran=pd.concat([x_data_n.iloc[:,0:5], x_data_n_nor], axis=1)
            Regressor,Encoder,X_train_t,mask_gamma,mask_delta,mask_upsilon,mask_omega,best_models=train_CFR(data_train_tran,y_data_tr,tol)
          
            
            
            #_______________loading best models
           
            Enc=TarNet(input_features,hid_enc,lat_dim,.1)
            Enc.to(device)
            Reg=Regressors(lat_dim*2,hid_y,.1)
            Reg.to(device)
            tclas=Tclassifier(lat_dim*2,.1)
            tclas.to(device)
            #____________________________
    
            
            for idx, (val_loss, filename) in enumerate(best_models):
            #print(f"Loading model {idx+1}: {filename} (val_loss={val_loss:.4f})")
        
                # Load saved state dict
                model_state = torch.load(filename) #map_location='cpu'
                Enc.load_state_dict(model_state['encoder'])
                Reg.load_state_dict(model_state['regressor'])
                tclas.load_state_dict(model_state['classifier'])
            
                data,y=get_data('test',i)
                data_n=add_dummy_features_shuffle(data,dummies)
                
                
                data_n_nor= pd.DataFrame(scaler_x.transform(data_n.iloc[:,5:]))
                data_test_tran=pd.concat([data_n.iloc[:,0:5], data_n_nor], axis=1)
                
    
                pehe_l.append(cal_pehe(data_test_tran,y,Reg,Enc,mask_gamma,mask_delta,mask_upsilon))
            ##############################
           
            
            print('files completed: ',i)
            gamma_dim_count.append(np.mean(gamma_count[-5:]))
            delta_dim_count.append(np.mean(delta_count[-5:]))
            upsilon_dim_count.append(np.mean(upsilon_count[-5:]))
            omega_dim_count.append(np.mean(omega_count[-5:]))
            #clear()
            torch.cuda.empty_cache()
            pehe_final.append(np.mean(pehe_l))
            clear()
    
    
        print('Tolerance',tol)
        print('Dummies shuffling: ',d)
        print('PEHE mean', np.mean(pehe_final))
        print('PEHE std', np.std(pehe_final))
        #print('KL count', np.mean(kl_avg))
        
        

        print('Gamma dim on avg',np.mean(gamma_dim_count))
        print('Delta dim on avg',np.mean(delta_dim_count))
        print('Upsilon dim on avg',np.mean(upsilon_dim_count))
        print('Omega dim on avg',np.mean(omega_dim_count))
        
       

        
       
        print('*************************************************')
    print('___________________________Next Tolerance_______________________________')

latent dimensions 105


/home/ahmad/envs/RLC/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
  0%|         | 0/8000 [00:00<?, ?it/s]/home/ahmad/envs/RLC/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|█| 8000/8000 [01:46<00:00, 74.94it

files completed:  1
Tolerance 0.4
Dummies shuffling:  0
PEHE mean 0.8670036196708679
PEHE std 0.0
Gamma dim on avg 6.8
Delta dim on avg 11.0
Upsilon dim on avg 8.6
*************************************************
___________________________Next Tolerance_______________________________
